In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import plotnine as gg

from regression.linear_model import LinearModel

In [ ]:
rng = np.random.default_rng(1)
data = pd.DataFrame({
    'x1':rng.normal(size=100).cumsum(),
    'x2':rng.normal(size=100).cumsum()
}).assign(y=lambda dt:rng.normal(1+2*dt.x1+3*dt.x2,scale=1))

formula = 'y~x1*x2'
mod=LinearModel(formula,data=data).fit(method='OLS')
mod.bootstrap_coef(n_resamples=1000)

mod.get_coef()

In [ ]:
mod.get_metric(bootstrap=True).round(4)

In [ ]:
ols(formula,data=data).fit().summary()

In [ ]:
(
    mod.predict(ci_method='bootstrap',alpha=0.05)
    .pipe(gg.ggplot)
    + gg.aes(x='x1')
    # + gg.geom_point(gg.aes(y='y'))
    + gg.geom_line(gg.aes(y='pred'),color='red')
    + gg.geom_ribbon(gg.aes(ymin='pred_low',ymax='pred_up'),alpha=0.3)
    + gg.geom_ribbon(gg.aes(ymin='ci_low',ymax='ci_up'),alpha=0.3,fill='red')
    # + gg.stat_function(fun=lambda x:np.sin(x),color='blue')
)

In [ ]:
(
    mod.coef_dist_boot
    .melt()
    .pipe(gg.ggplot)
    + gg.aes(x='value')
    + gg.geom_density()
    + gg.facet_wrap('variable',ncol=1,scales='free_y')
)

In [ ]:
(
    mod.coef_dist_boot
    .pipe(gg.ggplot)
    + gg.aes(x='Intercept',y='x1')
    + gg.geom_pointdensity()
    + gg.geom_density_2d()
)